In [11]:
import undetected_chromedriver as uc 
import os
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By

In [3]:
""" Test avec Seloger"""
pages = []


## Set Chrome Options
options = uc.ChromeOptions()
#options.headless=True
#options.add_argument('--headless')
#options.add_argument(f'--proxy-server={PROXY}')
#options.add_argument(f"user-agent={my_user_agent}")

## Create Undetected Chromedriver with Proxy
driver = uc.Chrome(options=options)


url = "https://www.seloger.com/list.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_refine-redirection-search_results"
## Send Request
driver.get(url)
annonces = driver.find_elements(By.CSS_SELECTOR, '[data-testid="sl.explore.coveringLink"]')
liste_liens = []
# Récupérez les liens des annonces
for lien in annonces:
    lien_annonce = lien.get_attribute('href')
    liste_liens.append(lien_annonce)
    print(lien_annonce)

driver.quit()



https://www.seloger.com/annonces/locations/appartement/strasbourg-67/koenigshoffen-ouest-centre-est/196415433.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/strasbourg-67/esplanade-nord-est/209798709.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/strasbourg-67/kable-nord/213792891.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/strasbourg-67/cronenbourg-est-centre-est/214331275.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_

In [4]:
liste_liens[3]

'https://www.seloger.com/annonces/locations/appartement/strasbourg-67/cronenbourg-est-centre-est/214331275.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S'

In [10]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

results = pd.DataFrame()

options = uc.ChromeOptions()
driver = uc.Chrome(options=options) 

for i in range(5):
    url_annonce = liste_liens[i]
    driver.get(url_annonce)

    try:
        # Utilisez une attente explicite pour attendre que la page accepter les cookies s'affiche
        continuer_sans_accepter_button = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//span[@class='didomi-continue-without-agreeing']"))
        )

        #Continue sans accepter les cookies
        continuer_sans_accepter_button[0].click()

        # Wait a bit for the text to fully load after clicking
        time.sleep(2)
            
    except Exception as e:
        print(f"Could not click 'Continuer sans accepter button: {e}")


    try: 
        voir_tout_button = driver.find_element(By.XPATH, "//button[@data-test='show-detail-feature-button-desktop']")
        voir_tout_button.click()
        time.sleep(1)

    except Exception as e:
        print(f"Could not click 'Voir toutes les caractéristiques' button: {e}")

    #Titre de l'annonce:
    titre_annonce = driver.find_elements(By.XPATH, '//div[@class="Summarystyled__Title-sc-1u9xobv-4 dbveQQ"]')[0].text


    #Prix:
    prix = driver.find_elements(By.XPATH, '//span[@class="global-styles__TextNoWrap-sc-1gbe8ip-6 gxurQr"]')[1].text #à voir si c'est toujours la 2e span 


    #Autres infos:
    #Ordre: Nb pièces, (Nb chambres), Surface, Nb etages, Balcon, Terasse, Jardin, Garage, Ascenceur, Piscine  ?Parking?
    autres_infos_WebElement = driver.find_elements(By.XPATH, '//div[@class="Tags__TagContainer-sc-edpl7u-0 EPxew"]')
    autres_infos = [element.text for element in autres_infos_WebElement]

    #Quartier:
    quartier = driver.find_elements(By.XPATH, '//span[@data-test="neighbourhood"]')[0].text

    #Ville + code postal:
    ville_cp = driver.find_elements(By.XPATH, '//span[@class="Localizationstyled__City-sc-gdkcr2-1 bgtLnh"]')[0].text


    #Description:
    description = driver.find_elements(By.XPATH, "//div[@class='ShowMoreText__UITextContainer-sc-1swit84-0 fDeZMv']")[0].text

    data_dict = {
        "Titre de l'annonce": [titre_annonce],
        "Prix": [prix],
        "Autres Infos": [", ".join(autres_infos)],
        "Quartier": [quartier],
        "Ville et CP": [ville_cp],
        "Description": [description]
    }


    #Caractéristiques:
    #Récupère les blocs de caractéristiques (ex: Extérieur, Cadre et situation...)
    blocs_caract = driver.find_elements(By.CSS_SELECTOR, "div.TitledDescription__TitledDescriptionContainer-sc-p0zomi-0")


    for bloc in blocs_caract:
        titre_caract_l = bloc.find_elements(By.CSS_SELECTOR, "div.feature-title") 
        if not titre_caract_l: #Vérifie si titre_caract est vide
            continue
        titre_caract = titre_caract_l[0].text #Récupère le titre des caractéristiques
        caract_list = bloc.find_elements(By.CSS_SELECTOR, "div.GeneralFeaturesstyled__TextWrapper-sc-1ia09m5-3")
        data_dict[titre_caract] = [", ".join([caract.text for caract in caract_list])]
        

    df=pd.DataFrame(data_dict)
    results = pd.concat([results,df], ignore_index=True)
    time.sleep(2)

driver.quit()









Could not click 'Voir toutes les caractéristiques' button: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@data-test='show-detail-feature-button-desktop']"}
  (Session info: chrome=121.0.6167.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   undetected_chromedriver             0x0000000101424168 undetected_chromedriver + 4673896
1   undetected_chromedriver             0x000000010141b9c3 undetected_chromedriver + 4639171
2   undetected_chromedriver             0x000000010100ffdd undetected_chromedriver + 397277
3   undetected_chromedriver             0x000000010105bbfc undetected_chromedriver + 707580
4   undetected_chromedriver             0x000000010105bdd1 undetected_chromedriver + 708049
5   undetected_chromedriver             0x00000001010a0284 undetected_chromedriver + 987780
6   undetected_chromedriver             0x000000

IndexError: list index out of range

In [6]:
results

,Titre de l'annonce,Prix,Autres Infos,Quartier,Ville et CP,Description,Extérieur,Cadre et situation,Services et accessibilité,Hygiène,Surfaces annexes,Cuisine,Pièces à vivre
0,Appartement à louer,751 €,"2 pièces, 1 chambre, 71 m², Étage 12/–",Quartier Canardiere Ouest Est,Strasbourg (67100),Spacieux appartement 2 pièces de 71 M² sur STR...,Balcon,Belle vue,Ascenseur,Salle de bain (baignoire),NaN,NaN,NaN
1,Appartement meublé à louer,1 400 €,"3 pièces, 2 chambres, 85 m², Étage 6/–",Quartier Robertsau Centre,Strasbourg (67000),Appartement 3 pièces idéalement situé ! Au 6 è...,Terrasse,NaN,Ascenseur,Salle de bain (baignoire),Cave,Cuisine séparée équipée,"Séjour / salon, Salle à manger, Entrée séparée"
2,Appartement à louer,900 €,"3 pièces, 2 chambres, 70 m², Étage 3/–",Quartier Hautepierre Centre,Strasbourg (67200),"Bel appartement situé au Rue Alfred de Vigny, ...",Balcon,NaN,Ascenseur,NaN,Garage,NaN,"Séjour / salon, Entrée séparée, Placards / ran..."
3,Appartement meublé à louer,662 €,"2 pièces, 1 chambre, 42 m², Étage 2/–",Quartier Forêt Noire Centre,Strasbourg (67000),Appartement meublée et confortable en location...,NaN,"Sans vis-à-vis, Belle vue","Ascenseur, Digicode, Interphone",NaN,NaN,NaN,NaN
4,Appartement meublé à louer,1 450 €,"3 pièces, 2 chambres, 85 m², Étage 5/5",Quartier Vauban Est,Strasbourg (67000),"Bel appartement meublé, entièrement rénové, id...",NaN,NaN,Pas d'ascenseur,Salle d'eau (douche),NaN,Cuisine équipée,NaN


In [11]:
results = pd.DataFrame()

In [7]:
def retrieve_data(url_annonce):
    options = uc.ChromeOptions()
    driver = uc.Chrome(options=options)
    driver.get(url_annonce)

    try:
        # Utilisez une attente explicite pour attendre que la page accepter les cookies s'affiche
        continuer_sans_accepter_button = WebDriverWait(driver, 100).until(
            EC.presence_of_all_elements_located((By.XPATH, "//span[@class='didomi-continue-without-agreeing']"))
        )

        #Continue sans accepter les cookies
        continuer_sans_accepter_button[0].click()

        # Wait a bit for the text to fully load after clicking
        time.sleep(2)
            
    except Exception as e:
        print(f"Could not click 'Continuer sans accepter button: {e}")


    try: 
        voir_tout_button = driver.find_element(By.XPATH, "//button[@data-test='show-detail-feature-button-desktop']")
        voir_tout_button.click()
        time.sleep(1)

    except Exception as e:
        print(f"Could not click 'Voir toutes les caractéristiques' button: {e}")

    #Titre de l'annonce:
    titre_annonce = driver.find_elements(By.XPATH, '//div[@class="Summarystyled__Title-sc-1u9xobv-4 dbveQQ"]')[0].text


    #Prix:
    prix = driver.find_elements(By.XPATH, '//span[@class="global-styles__TextNoWrap-sc-1gbe8ip-6 gxurQr"]')[1].text #à voir si c'est toujours la 2e span 


    #Autres infos:
    #Ordre: Nb pièces, (Nb chambres), Surface, Nb etages, Balcon, Terasse, Jardin, Garage, Ascenceur, Piscine  ?Parking?
    autres_infos_WebElement = driver.find_elements(By.XPATH, '//div[@class="Tags__TagContainer-sc-edpl7u-0 EPxew"]')
    autres_infos = [element.text for element in autres_infos_WebElement]

    #Quartier:
    quartier = driver.find_elements(By.XPATH, '//span[@data-test="neighbourhood"]')[0].text

    #Ville + code postal:
    ville_cp = driver.find_elements(By.XPATH, '//span[@class="Localizationstyled__City-sc-gdkcr2-1 bgtLnh"]')[0].text


    #Description:
    description = driver.find_elements(By.XPATH, "//div[@class='ShowMoreText__UITextContainer-sc-1swit84-0 fDeZMv']")[0].text

    data_dict = {
        "Titre de l'annonce": [titre_annonce],
        "Prix": [prix],
        "Autres Infos": [", ".join(autres_infos)],
        "Quartier": [quartier],
        "Ville et CP": [ville_cp],
        "Description": [description]
    }


    #Caractéristiques:
    #Récupère les blocs de caractéristiques (ex: Extérieur, Cadre et situation...)
    blocs_caract = driver.find_elements(By.CSS_SELECTOR, "div.TitledDescription__TitledDescriptionContainer-sc-p0zomi-0")


    for bloc in blocs_caract:
        titre_caract_l = bloc.find_elements(By.CSS_SELECTOR, "div.feature-title") 
        if not titre_caract_l: #Vérifie si titre_caract est vide
            continue
        titre_caract = titre_caract_l[0].text #Récupère le titre des caractéristiques
        caract_list = bloc.find_elements(By.CSS_SELECTOR, "div.GeneralFeaturesstyled__TextWrapper-sc-1ia09m5-3")
        data_dict[titre_caract] = [", ".join([caract.text for caract in caract_list])]
        

    df2=pd.DataFrame(data_dict)
    print(df2)
    driver.quit()

Preprocessing:

In [81]:
import re
import pandas as pd 
# Fonction pour extraire les chiffres
def extract_number(s):
    # Si 's' est déjà un nombre (int ou float), retournez-le directement
    if isinstance(s, (int, float)):
        return s
    # Sinon, c'est une chaîne et on tente d'extraire le nombre
    # Cette regex prend en compte les nombres avec et sans séparateurs de milliers, et sans signe de devise
    match = re.search(r'(\d{1,3}(?:\s?\d{3})*)', str(s))
    # Retirez les espaces pour convertir la séquence en un entier
    return int(''.join(match.group(1).split())) if match else None

def extract_after_etage(s):
    # Convertir l'entrée en une chaîne si ce n'est pas déjà le cas
    s = str(s)
    
    # Vérifier si la chaîne est déjà au format X/Y ou X/-
    already_formatted_match = re.match(r'^\d+[/\-]\d*$', s)
    if already_formatted_match:
        return s  # La chaîne est déjà au format désiré, donc on la retourne directement

    # Pattern regex pour trouver et capturer tout après "Étage "
    etage_match = re.search(r'Étage\s*(.*)', s)
    if etage_match:
        return etage_match.group(1)  # Retourne la correspondance trouvée après "Étage"
    
    # Si aucune des conditions ci-dessus n'est remplie, retourne None
    return None

def extract_surface(text, surface_type):
    if pd.isnull(text):
        return None
    pattern = re.compile(rf'{surface_type} (\d+) m²', re.IGNORECASE)
    match = pattern.search(text)
    if match:
        return int(match.group(1))  # Le groupe 1 est maintenant le nombre capturé
    else:
        return None
    
def extract_exposition(text):
    if pd.isnull(text):
        return None
    # Recherche d'un motif où "Exposition" est suivi par n'importe quels caractères jusqu'à une virgule
    match = re.search(r'Exposition\s([^,]+),', text)
    if match:
        return match.group(1)  # Le groupe 1 contient la partie de la chaîne entre "Exposition " et la première virgule
    else:
        return None
    
def check_ascenseur(value):
    if pd.isnull(value):
        return None
    if 'Pas d\'ascenseur' in value:
        return 0
    if 'Ascenseur : non communiqué' in value or 'Ascenseur' not in value:
        return None
    if 'Ascenseur' in value:
        return 1
    

In [82]:
df = pd.read_csv('output.csv')
df


,title,price,city and zip code,neighbourhood,nb_rooms,nb_bedrooms,surface,numero_etage,description,Extérieur,Cadre et situation,Surfaces annexes,Services et accessibilité,Cuisine,Hygiène,Pièces à vivre,Diagnostic de performance énergétique (DPE),Indice d'émission de gaz à effet de serre (GES),loyer_base,charges_forfaitaires,complement_loyer,depot_garantie,loyer_charges_comprises
0,Appartement à louer,975 €,Strasbourg (67000),Quartier Petite France Centre,2 pièces,1 chambre,54 m²,Étage 5/6,APPARTEMENT 2 PIÈCES AVEC BALCON - DERNIER ÉTA...,Balcon 4 m²,"Exposition Nord/Sud, Belle vue",Cave,"Ascenseur, Interphone",Cuisine séparée équipée,"Salle de bain (baignoire), Toilettes, Toilette...",NaN,D,E,NaN,NaN,NaN,NaN,975 € / mois
1,Studio meublé à louer,478 €,Strasbourg (67000),Quartier Petite France Centre,1 pièce,NaN,19 m²,Étage 2/–,"Au centre ville de Strasbourg, dans le quartie...",NaN,NaN,NaN,"Ascenseur, Gardien",Coin cuisine,"Salle d'eau (douche), Toilettes","Entrée séparée, Placards / rangements",Diagnostic de performance énergétique (DPE) no...,Indice d'émission de gaz à effet de serre (GES...,NaN,NaN,NaN,NaN,478 € / mois
2,Appartement à louer,1 027 €,Strasbourg (67100),Quartier Stockfeld Est,4 pièces,3 chambres,103 m²,Étage 3/5,A LOUER - Appartement 5 pièces - 103m² - rue G...,Balcon,Belle vue,NaN,Ascenseur,NaN,NaN,Parquet,D,D,NaN,NaN,NaN,NaN,1 027 € / mois
3,Appartement à louer,884 €,Strasbourg (67300),Quartier Robertsau Centre,3 pièces,2 chambres,65 m²,Étage 3/–,"A 10 mn du centre ville de Strasbourg, au coeu...",Terrasse,Calme,Parking,Ascenseur,NaN,Salle de bain (baignoire),"Séjour / salon, Placards / rangements",Diagnostic de performance énergétique (DPE) no...,Indice d'émission de gaz à effet de serre (GES...,NaN,NaN,NaN,NaN,884 € / mois
4,Appartement meublé à louer,1 091 €,Strasbourg (67000),Quartier Petite France Centre,3 pièces,2 chambres,61 m²,Étage 3/–,"Dans l'hyper centre de Strasbourg, a proximité...",Balcon,NaN,NaN,"Ascenseur, Interphone",Cuisine séparée équipée,"Salle de bain (baignoire), Toilettes","Entrée séparée, Placards / rangements, Parquet",F,E,NaN,NaN,NaN,NaN,1 091 € / mois
5,Appartement à louer,1 605 €,Strasbourg (67100),Quartier Krutenau Centre Est,4 pièces,3 chambres,104 m²,Étage 2/2,ROUTE DU POLYGONE - Idéal colocation\r\n\r\nAp...,NaN,NaN,Garage,Pas d'ascenseur,Cuisine équipée,"Salle de bain (baignoire), Toilettes",Séjour / salon 50 m²,C,D,NaN,NaN,NaN,NaN,1 605 € / mois
6,Appartement meublé à louer,700 €,Strasbourg (67100),Quartier Plaine des Bouchers-Centre,2 pièces,NaN,40 m²,Étage 0/0,"2 pièces meublé 39.9m², STRASBOURG MEINAU - Ro...",Terrasse,NaN,Parking,Ascenseur : non communiqué,Cuisine équipée,Salle d'eau (douche),NaN,F,C,NaN,NaN,NaN,NaN,700 € / mois
7,Appartement à louer,860 €,Strasbourg (67000),Quartier Plaine des Bouchers-Centre,3 pièces,2 chambres,64 m²,Étage 2/–,"A 10 min du centre ville de STRASBOURG, à prox...",Balcon,NaN,NaN,Ascenseur,Cuisine équipée,"Salle de bain (baignoire), Toilettes, Toilette...","Séjour / salon, Salle à manger",Diagnostic de performance énergétique (DPE) no...,Indice d'émission de gaz à effet de serre (GES...,NaN,NaN,NaN,NaN,860 € / mois
8,Studio meublé à louer,660 €,Strasbourg (67000),Quartier Esplanade Nord Est,1 pièce,NaN,15 m²,NaN,[FR] Comment fonctionne ?\r\n est une platefor...,NaN,Calme,NaN,Ascenseur,NaN,NaN,Entrée séparée,Diagnostic de performance énergétique (DPE) no...,Indice d'émission de gaz à effet de serre (GES...,NaN,NaN,NaN,NaN,660 € / mois
9,Appartement à louer,760 €,Strasbourg (67000),Quartier Forêt Noire Centre,4 pièces,3 chambres,64 m²,Étage 5/5,COLOCATION - APPARTEMENT 4 PIÈCES - STRASBOURG...,NaN,NaN,NaN,"Pas d'ascenseur, Interphone",Cuisine équipée,"Salle d'eau (douche), Toilettes",Séjour / salon,F,F,NaN,NaN,NaN,NaN,760 € / mois


In [84]:
df['type'] = df['title'].apply(lambda x: x.split()[0])
df['meublé'] = df['title'].apply(lambda x: 1 if "meublé" in x else 0)

# Séparation de la colonne "city and zip code"
df[['city', 'zipcode']] = df['city and zip code'].str.split('(', expand=True)
df['zipcode'] = df['zipcode'].str.replace(')', '')
df['nb_rooms'] = df['nb_rooms'].apply(lambda x : extract_number(x))
df['nb_rooms'] = df['nb_rooms'].astype('Int64')
df['nb_bedrooms'] = df['nb_bedrooms'].apply(lambda x : extract_number(x))
df['nb_bedrooms'] = df['nb_bedrooms'].astype('Int64')
df['surface'] = df['surface'].apply(lambda x : extract_number(x))
df['numero_etage'] = df['numero_etage'].apply(extract_after_etage)
df['balcon'] = df['Extérieur'].apply(lambda x: 1 if pd.notnull(x) and 'Balcon' in x else 0)
df['terrasse'] = df['Extérieur'].apply(lambda x: 1 if pd.notnull(x) and 'Terrasse' in x else 0)
df['jardin'] = df['Extérieur'].apply(lambda x: 1 if pd.notnull(x) and 'Jardin' in x else 0)
df['surface_balcon'] = df['Extérieur'].apply(lambda x: extract_surface(x, "Balcon"))
df['surface_terrasse'] = df['Extérieur'].apply(lambda x: extract_surface(x, "Terrasse"))
df['surface_jardin'] = df['Extérieur'].apply(lambda x: extract_surface(x, "Jardin"))
df['exposition'] = df['Cadre et situation'].apply(lambda x: extract_exposition(x))
df['meublé'] = df['title'].apply(lambda x: 1 if "meublé" in x else 0)
df['cave'] = df['Surfaces annexes'].apply(lambda x: 1 if pd.notnull(x) and 'Cave' in x else 0)
df['parking'] = df['Surfaces annexes'].apply(lambda x: 1 if pd.notnull(x) and 'Parking' in x else 0)
df['garage'] = df['Surfaces annexes'].apply(lambda x: 1 if pd.notnull(x) and 'Garage' in x else 0)
df['box'] = df['Surfaces annexes'].apply(lambda x: 1 if pd.notnull(x) and 'Box' in x else 0)
df['ascenseur'] = df['Services et accessibilité'].apply(check_ascenseur)
df['ascenseur'] = df['ascenseur'].astype('Int64')
df['interphone'] = df['Services et accessibilité'].apply(lambda x: 1 if pd.notnull(x) and 'Interphone' in x else 0)
df['gardien'] = df['Services et accessibilité'].apply(lambda x: 1 if pd.notnull(x) and 'Gardien' in x else 0)
df['salle de bain (Baignoire)'] = df['Hygiène'].apply(lambda x: 1 if pd.notnull(x) and 'baignoire' in x else 0)
df['salle d\'eau (douche)'] = df['Hygiène'].apply(lambda x: 1 if pd.notnull(x) and 'douche' in x else 0)
df['surface_salon'] = df['Pièces à vivre'].apply(lambda x: extract_surface(x, "Séjour / salon"))
df['surface_salle_a_manger'] = df['Pièces à vivre'].apply(lambda x: extract_surface(x, "Salle à manger"))
df['loyer_charges_comprises'] = df['loyer_charges_comprises'].apply(lambda x : extract_number(x))

pd.set_option('display.max_columns', None)
df

,title,price,city and zip code,neighbourhood,nb_rooms,nb_bedrooms,surface,numero_etage,description,Extérieur,Cadre et situation,Surfaces annexes,Services et accessibilité,Cuisine,Hygiène,Pièces à vivre,Diagnostic de performance énergétique (DPE),Indice d'émission de gaz à effet de serre (GES),loyer_base,charges_forfaitaires,complement_loyer,depot_garantie,loyer_charges_comprises,type,meublé,city,zipcode,balcon,terrasse,jardin,surface_balcon,surface_terrasse,surface_jardin,exposition,cave,parking,garage,box,ascenseur,interphone,gardien,salle de bain (Baignoire),salle d'eau (douche),surface_salon,surface_salle_a_manger
0,Appartement à louer,975 €,Strasbourg (67000),Quartier Petite France Centre,2,1,54.0,5/6,APPARTEMENT 2 PIÈCES AVEC BALCON - DERNIER ÉTA...,Balcon 4 m²,"Exposition Nord/Sud, Belle vue",Cave,"Ascenseur, Interphone",Cuisine séparée équipée,"Salle de bain (baignoire), Toilettes, Toilette...",NaN,D,E,NaN,NaN,NaN,NaN,975.0,Appartement,0,Strasbourg,67000,1,0,0,4.0,None,None,Nord/Sud,1,0,0,0,1,1,0,1,0,NaN,NaN
1,Studio meublé à louer,478 €,Strasbourg (67000),Quartier Petite France Centre,1,<NA>,19.0,None,"Au centre ville de Strasbourg, dans le quartie...",NaN,NaN,NaN,"Ascenseur, Gardien",Coin cuisine,"Salle d'eau (douche), Toilettes","Entrée séparée, Placards / rangements",Diagnostic de performance énergétique (DPE) no...,Indice d'émission de gaz à effet de serre (GES...,NaN,NaN,NaN,NaN,478.0,Studio,1,Strasbourg,67000,0,0,0,NaN,None,None,None,0,0,0,0,1,0,1,0,1,NaN,NaN
2,Appartement à louer,1 027 €,Strasbourg (67100),Quartier Stockfeld Est,4,3,103.0,3/5,A LOUER - Appartement 5 pièces - 103m² - rue G...,Balcon,Belle vue,NaN,Ascenseur,NaN,NaN,Parquet,D,D,NaN,NaN,NaN,NaN,1027.0,Appartement,0,Strasbourg,67100,1,0,0,NaN,None,None,None,0,0,0,0,1,0,0,0,0,NaN,NaN
3,Appartement à louer,884 €,Strasbourg (67300),Quartier Robertsau Centre,3,2,65.0,None,"A 10 mn du centre ville de Strasbourg, au coeu...",Terrasse,Calme,Parking,Ascenseur,NaN,Salle de bain (baignoire),"Séjour / salon, Placards / rangements",Diagnostic de performance énergétique (DPE) no...,Indice d'émission de gaz à effet de serre (GES...,NaN,NaN,NaN,NaN,884.0,Appartement,0,Strasbourg,67300,0,1,0,NaN,None,None,None,0,1,0,0,1,0,0,1,0,NaN,NaN
4,Appartement meublé à louer,1 091 €,Strasbourg (67000),Quartier Petite France Centre,3,2,61.0,None,"Dans l'hyper centre de Strasbourg, a proximité...",Balcon,NaN,NaN,"Ascenseur, Interphone",Cuisine séparée équipée,"Salle de bain (baignoire), Toilettes","Entrée séparée, Placards / rangements, Parquet",F,E,NaN,NaN,NaN,NaN,1091.0,Appartement,1,Strasbourg,67000,1,0,0,NaN,None,None,None,0,0,0,0,1,1,0,1,0,NaN,NaN
5,Appartement à louer,1 605 €,Strasbourg (67100),Quartier Krutenau Centre Est,4,3,104.0,2/2,ROUTE DU POLYGONE - Idéal colocation\r\n\r\nAp...,NaN,NaN,Garage,Pas d'ascenseur,Cuisine équipée,"Salle de bain (baignoire), Toilettes",Séjour / salon 50 m²,C,D,NaN,NaN,NaN,NaN,1605.0,Appartement,0,Strasbourg,67100,0,0,0,NaN,None,None,None,0,0,1,0,0,0,0,1,0,50.0,NaN
6,Appartement meublé à louer,700 €,Strasbourg (67100),Quartier Plaine des Bouchers-Centre,2,<NA>,40.0,0/0,"2 pièces meublé 39.9m², STRASBOURG MEINAU - Ro...",Terrasse,NaN,Parking,Ascenseur : non communiqué,Cuisine équipée,Salle d'eau (douche),NaN,F,C,NaN,NaN,NaN,NaN,700.0,Appartement,1,Strasbourg,67100,0,1,0,NaN,None,None,None,0,1,0,0,<NA>,0,0,0,1,NaN,NaN
7,Appartement à louer,860 €,Strasbourg (67000),Quartier Plaine des Bouchers-Centre,3,2,64.0,None,"A 10 min du centre ville de STRASBOURG, à prox...",Balcon,NaN,NaN,Ascenseur,Cuisine équipée,"Salle de bain (baignoire), Toilettes, Toilette...","Séjour / salon, Salle à manger",Diagnostic de performance énergétique (DPE) no...,Indice d'émission de gaz à effet de serre (GES...,NaN,NaN,NaN,NaN,860.0,Appartement,0,Strasbourg,67000,1,0,0,NaN,None,None,None,0,0,0,0,1,0,0,1,0,NaN,NaN
8,Studio meublé à louer,660 €,Strasbourg (67000),Quartier Esplanade Nord Est,1,<NA>,15.0,None,[FR] Comment fonctionne ?\r\n est une platefor...,NaN,Calme,Na